In [1]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
#import glob
from pandas import DataFrame
from pandas.io.parsers import TextFileReader
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
import itertools

In [2]:
gau = pd.read_csv('./result/result_matrix_ie_gau.txt',sep='\t',index_col=0,header=0)
exp = pd.read_csv('./result/result_matrix_ie_toni_exp.txt',sep='\t',index_col=0,header=0)
path = pd.read_csv('./result/result_matrix_ie_toni_path.txt',sep='\t',index_col=0,header=0)
seq = pd.read_csv('./result/result_matrix_ie_seq.txt',sep='\t',index_col=0,header=0)

In [3]:
tf = pd.read_csv('./result/result_matrix_ie_ld_tf_0.95.txt',sep='\t',index_col=0,header=0)
gene = pd.read_csv('./result/result_matrix_ie_ld_gene_0.97.txt',sep='\t',index_col=0,header=0)

In [4]:
miRNA=pd.read_csv("./data/all_mirna.txt", sep="\t",header=None)
disease=pd.read_csv("./data/all_disease.txt", sep="\t",header=None)
causal = pd.read_csv("./data/yanzheng2.txt", header=None, delimiter="\t",names=["causal","mir","dis","yn",])

In [5]:
testset =  pd.read_csv("./data/hmdd_causal_test.txt",sep="\t",header= None)
trainset =  pd.read_csv("./data/hmdd_causal_train.txt",header = None,sep="\t")
trainset.columns = ['mir','dis','cau']
testset.columns = ['mir','dis','cau']

In [6]:
weight_range = [i/100 for i in range(5, 76, 5)]  # 权重范围为0.05-0.75，步长为0.05
area_list=[]
weight_list = []
i_area_list=[]

matrices = [seq,exp,gau,path,gene,tf]

for weights in itertools.product(weight_range, repeat=len(matrices)):
    if sum(weights) == 1:  # 权重和为1
        # 计算整合后的矩阵MS
        Prediction = sum(w * matrix for w, matrix in zip(weights, matrices))
        
        file = open("result/Result_score.txt","w")
        for x in range(len(miRNA)):
            for y in range(len(disease)):
                file.write(str(x)+"\t"+str(y)+"\t"+miRNA.iloc[x,1]+"\t"+disease.iloc[y,1]+"\t"+str(Prediction.iloc[x,y])+"\n")
        file.close()

        result = pd.read_csv("result/Result_score.txt", header=None, delimiter="\t",names=["nmir","ndis","mir","dis","prediction"])
        c_result = pd.merge(result,causal,on=['mir','dis'],how = 'left')
        r = c_result[["mir","dis","prediction","causal"]]
        data = r.fillna(3)
        data = data.loc[data["causal"] != 3]
        y_true = np.array(data.iloc[:,3])
        y_true = np.where(y_true == 1,True,False)
        y_scores = np.array(data.iloc[:,2])
        area = 0
        fpr,tpr,threshold = roc_curve(y_true,y_scores)
        area = metrics.auc(fpr,tpr)
        area_list.append(area)
        weight_list.append(weights)
        
        a = pd.merge(result,testset,on=['mir','dis'],how = 'left')
        data_i = a.fillna(0)
        cau = np.array(data_i.iloc[:,5])
        cas = np.where(cau == 1,True,False)
        prediction = np.array(data_i.iloc[:,4])
        i_area = 0
        fpr,tpr,threshold = roc_curve(cas,prediction)
        i_area = metrics.auc(fpr,tpr)
        i_area_list.append(i_area)
        

In [7]:
print(len(area_list))
print(len(i_area_list))
print(len(weight_list))

9937
9937
9937


In [8]:
quanzhong_toni = pd.DataFrame({'weight':weight_list, 'causal_ROC':area_list, 'independent_ROC':i_area_list})

In [9]:
quanzhong_toni_sort = quanzhong_toni.sort_values(by='causal_ROC',ascending = False)

In [10]:
quanzhong_toni_sort.to_csv('./result/quanzhong_toni.txt',sep='\t')

In [18]:
area_list = []
location_list = []
weights = np.arange(0.05,0.90,0.05)

miRNA=pd.read_csv("./data/all_mirna.txt", sep="\t",header=None)
disease=pd.read_csv("./data/all_disease.txt", sep="\t",header=None)
causal = pd.read_csv("./data/yanzheng2.txt", header=None, delimiter="\t",names=["causal","mir","dis","yn",])
print(len(miRNA))
print(len(disease))

535
302
